# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [5]:
! git clone https://github.com/XaxUser/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

Cloning into 'exam_2025'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 59 (delta 21), reused 20 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 1.41 MiB | 15.01 MiB/s, done.
Resolving deltas: 100% (21/21), done.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [6]:
mySeed = 200

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [7]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

### Réponse
Pour estimer les coefficients $\theta_k$ on peut utiliser la régression linéaire.

In [11]:
from utils_exercices import generate_dataset
from sklearn.linear_model import LinearRegression

# Générer les jeux de données
train_set, test_set = generate_dataset(mySeed)

# Extraction des données d'entraînement
X_train = train_set['inputs']  # Matrice des prédicteurs
y_train = train_set['targets']  # Vecteur des cibles

# Extraction des données de test (optionnel, utilisé pour validation)
X_test = test_set['inputs']
y_test = test_set['targets']

# Initialiser et ajuster le modèle de régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)

# Récupérer les coefficients et l'ordonnée à l'origine
theta_0 = model.intercept_  # Ordonnée à l'origine
coefficients = model.coef_  # Coefficients associés aux prédicteurs

# Affichage des résultats
print("Résultats de la régression linéaire :")
print(f"Ordonnée à l'origine (theta_0): {theta_0}")
print(f"Coefficients des prédicteurs (theta_1, theta_2, theta_3): {coefficients}")


Résultats de la régression linéaire :
Ordonnée à l'origine (theta_0): 10.078764034363882
Coefficients des prédicteurs (theta_1, theta_2, theta_3): [1.95156862 1.94842221 3.59966699]


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

### Réponse
Pour estimer les $\theta_k$ à l’aide d’un réseau de neurones entraîné par SGD, l’architecture la plus adaptée est un réseau de neurones simple (simple perceptron) avec :
- Un seul neurone en sortie : Pour produire une prédiction $t$, en tant que combinaison linéaire des entrées $x$, $y$, $z$.

- Pas de couche cachée : Une seule couche linéaire suffit, car le problème est linéaire (donnée par la relation suivante) :
$
t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon.
$

- Fonction d’activation linéaire : Aucune activation n’est nécessaire pour un problème purement linéaire.

Justification:
- Expressivité : Un réseau à une seule couche (sans activation non-linéaire) est suffisant pour apprendre une relation linéaire. Ajouter des couches cachées serait inutile et pourrait ralentir l'entraînement.
- Performance en généralisation : Ce modèle simple a peu de paramètres et évite le surapprentissage. Il est donc efficace pour généraliser à des données non vues.

In [ ]:
# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

# A coder :
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # Couche linéaire pour apprendre les coefficients
        self.linear = nn.Linear(3, 1)  # 3 entrées (x, y, z), 1 sortie (t)
    def forward(self, x):
        return self.linear(x)  # Pas d'activation pour garder la linéarité

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [ ]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:

      ...

**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

In [ ]:
# Nb de paramètres dans self.Down1: (calcul "à la main")

# Nb de paramètres au total:

**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG?token=GHSAT0AAAAAAC427DACOPGNDNN6UDOLVLLAZ4BB2JQ" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?